In [ ]:
import os
import sys
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

import numpy as np
import time as time
import glob
import random
import constants

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)

MOCK_XCORR_DIR = os.path.join(constants.BIAS_DIR_BASE, 'xcorr', 'mock', 'crosscorr')

In [ ]:
X, Y = np.meshgrid(SigEdges, PiEdges)

SigMax = 35.
PiMin = -40.
PiMax = 40.

def plot_one_crosscorr(crosscorr_path, xcorr=None):
    if xcorr is None:
        xcorr = np.load(crosscorr_path).T

    plt.figure(figsize=(2, 14/3))
    plt.pcolormesh(X, Y, xcorr,cmap='jet_r',vmin=-0.2, vmax=0.1 )
    plt.gca().set_aspect('equal')
    plt.gca().set_xlim(np.min(X), SigMax)
    plt.gca().set_ylim(PiMin, PiMax)
    plt.gca().set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
    plt.gca().set_ylabel(r'$\pi\; (\mathrm{cMpc})$')

In [ ]:
# for p in random.sample(list(glob.glob(os.path.join(MOCK_XCORR_DIR, '*.npy'))), 100):
#     plot_one_crosscorr(p)
#     plt.show()

In [ ]:
def average_xcorrs(prefix):
    xcorr_path_list = glob.glob(os.path.join(MOCK_XCORR_DIR, f"{prefix}*"))
    xcorr_plot_sum = 0
    for i, p in enumerate(xcorr_path_list):
        xcorr_plot_sum += np.load(p).T
    return xcorr_plot_sum / len(xcorr_path_list)

In [ ]:
log_lb_list = np.arange(9, 13.4 - 0.1, 0.1)

for lb in log_lb_list:
    ub = lb + 0.1
    avg_xcorr = average_xcorrs(f'xcorrmock_survey_CLAMATO_mass_{lb}_{ub}')
    plot_one_crosscorr('', xcorr=avg_xcorr)
    plt.title(f'{lb} {ub}')
    plt.show()